# Kapitel 8: Datenaufbereitung - Verknüpfen und Umformen

McKinney, W. (2017). *Python for Data Analysis: Data Wrangling with Pandas, NumPy, and IPython*. 2. Auflage. Sebastopol, CA [u. a.]: O’Reilly.

Überarbeitet: armin.baenziger@zhaw.ch, 28. Februar 2020

- In vielen Anwendungen können Daten über mehrere Dateien oder Datenbanken verteilt sein oder in einer Form angeordnet sein, die nicht leicht zu analysieren ist. 
- Dieses Kapitel konzentriert sich auf Tools zum Kombinieren, Verknüpfen und Umformen von Daten.
- Zuerst führen wir das Konzept der hierarchischen Indexierung in Pandas ein, das in einigen dieser Operationen ausführlich verwendet wird. 
- Danach betrachten wir spezielle Datenmanipulationen. Verschiedene Anwendungen dieser Werkzeuge werden in Kapitel 14 (bzw. im Unterricht) vorgestellt, wo konkrete (grössere) Datensätze analysiert werden.



In [ ]:
%autosave 0

In [ ]:
# Bibliotheken mit üblichen Abkürzungen laden:
import numpy as np
import pandas as pd

## Hierarchische Indexierung
- Die hierarchische Indexierung ist ein Merkmal von Pandas, mit dem man mehrere (zwei oder mehr) Indexstufen auf einer Achse haben kann. 
- Etwas abstrakt bietet es eine Möglichkeit, mit *höherdimensionalen Daten* (z. B. drei Dimensionen) in einer niederdimensionalen Form (z. B. zwei Dimensionen; Zeilen und Spalten) zu arbeiten. 
- Wir beginnen mit einem einfachen Beispiel: Einer Series mit einer *Liste von Listen (oder Arrays)* als Index:

In [ ]:
# Beispieldaten generieren:
Jahr    = [2018]*4 + [2019]*4 + [2020]*2 
Quartal = [1, 2, 3, 4]*2 + [1, 2]

Umsatz = pd.Series([11, 13, 10, 19, 10, 12, 9, 21, 13, 15],
          index = [Jahr, Quartal])*10

Umsatz.index.names = ['Jahr', 'Quartal']

Umsatz

Beachten Sie den hierarchischen Index: Hierarchieebene 1 des Index sind die Jahre und Hierarchieebene 2 die Quartale.

Bei einem hierarchisch indizierten Objekt ist eine so genannte *partielle Indizierung* möglich, mit der Teilmengen der Daten übersichtlich ausgewählt werden können:

In [ ]:
# Alle Umsätze im 2019 (erste Hierarchieebene):
Umsatz.loc[2019]     

In [ ]:
# Alle Umsätze ab 2019:
Umsatz.loc[2019:] 

In [ ]:
# Alle Umsätze von 2018 und 2020:
Umsatz.loc[[2018, 2020]]

Es ist auch möglich, von einer "inneren" Ebene zu selektieren (hier 1. Quartal):

In [ ]:
Umsatz.loc[:, 1]  # Alle Umsätze im 1. Quartal

**Kontrollfragen:**

In [ ]:
# Gegeben:
Umsatz

In [ ]:
# Frage 1: Was ist der Output?
Umsatz.loc[2019].sum()

In [ ]:
# Frage 2: Was ist der Output?
Umsatz.loc[:, 2].sum()

- Die *hierarchische Indizierung* spielt eine wichtige Rolle bei der Umformung von Daten und gruppenbasierten Vorgängen wie der Bildung von *Pivot-Tabellen*. 
- Beispielsweise könnten die Daten mithilfe ihrer `unstack`-Methode in ein DataFrame umgeformt werden:

In [ ]:
# Zur Erinnerung:
Umsatz   # eine Series mit hierarchischem Index

In [ ]:
df = Umsatz.unstack()
# Die zweite (innere) Hierarchieebene wird in die Spalten
# "gedreht" (vom "Long"- ins "Wide"-Format).
df

Es entstehen zwei `NaN` am Ende, da die entsprechenden Quartalszahlen im letzten Jahr fehlen.  

Es wäre auch möglich gewesen, die erste Hierarchieebene in die Spalten zu drehen:

In [ ]:
Umsatz.unstack(level=0)

Die inverse Operation ist `stack`:

In [ ]:
df.stack()  # Stack "dreht" vom "Wide"- ins "Long"-Format

Bei einem DataFrame kann *jede Achse* (Zeilen oder Spalten) einen hierarchischen Index haben:

In [ ]:
df

In [ ]:
df.columns = [['Halbjahr_1']*2 + ['Halbjahr_2']*2, 
              [1, 2, 3, 4]]
df

In [ ]:
df.Halbjahr_2  # nur Spalten mit Überschrift "Halbjahr_2" auswählen

**Kontrollfrage:**

In [ ]:
# Gegeben:
ser = pd.Series([1, 3, 2, 3, 5], 
                index=[list('AABBB'), list('ababc')])
ser

In [ ]:
# Frage: Was ist der Output?
ser.unstack()

### Zusammenfassende Statistiken nach Hierarchieebene
Viele beschreibende und zusammenfassende Statistiken zu DataFrames und Series verfügen über eine **`level`-Option**, *mit der man die Hierarchieebene angeben kann, nach der auf einer bestimmten Achse aggregiert werden soll*.

In [ ]:
# Beispieldaten:
Umsatz_df = pd.DataFrame({'Filiale_A': [11, 13, 10, 19, 10, 12, 9, 21, 13, 15],
                          'Filiale_B': [9, 11, 8, 15, 10, 11, 10, 16, 11, 13]},
                          index = [Jahr, Quartal]) * 10

Umsatz_df.index.names = ['Jahr', 'Quartal']

Umsatz_df

In [ ]:
Umsatz_df.sum()   # Spaltensummen

In [ ]:
Umsatz_df.sum(level='Quartal')    # Spaltensummen nach 'Quartal'

**Kontrollfragen:**

In [ ]:
# Gegeben:
Umsatz_df

In [ ]:
# Frage 1: Was ist der Output?
Umsatz_df.sum(level='Jahr')

In [ ]:
# Frage 2: Was ist der Output?
Umsatz_df['Filiale_B'].mean(level='Jahr')

### Indexierung mit einer Spalte eines DataFrame
- Es ist nicht ungewöhnlich, dass man eine oder mehrere Spalten aus einem DataFrame als (Zeilen-) Index verwenden möchte. 
- Alternativ ist es möglich, den Zeilenindex in die Spalten des DataFrames zu verschieben. 
- Beispiel:

In [ ]:
# Beispieldaten:
Umsatz_Daten = Umsatz_df.reset_index()
Umsatz_Daten

In [ ]:
# Spalten "Jahr" und "Quartal" in den Index verschieben:
Umsatz_Daten.set_index(['Jahr', 'Quartal'], inplace=True)
Umsatz_Daten

**Kontrollfragen:**

In [ ]:
# Gegeben:
drinks = pd.read_csv('../weitere_Daten/drinksbycountry.csv', usecols=[0, 4, 5])
drinks.rename(columns={'total_litres_of_pure_alcohol': 'alcohol'}, inplace=True)
drinks5 = drinks.sample(5, random_state=6).reset_index(drop=True)  
drinks5

In [ ]:
# Frage 1: Was ist der Output?
drinks5HI = drinks5.set_index(['continent', 'country']).sort_index()
drinks5HI

In [ ]:
# Frage 2: Was ist der Output?
drinks5HI.loc['Asia']

In [ ]:
# Frage 3: Was ist der Output?
drinks5HI.loc[('Asia', 'Kuwait')]

## Kombinieren und Verknüpfen von Datensätzen
Wir folgen in diesem Abschnitt dem **"Cheat Sheet"** *"Data Wrangling with pandas"* statt dem Lehrmittel, das für den Einstieg zu viele Funktionalitäten bespricht, wie ich meine. Hier versuche ich Ihnen einen guten Überblick über die Möglichkeiten zu geben und bei konkreten Problemstellungen kann man sich dann mit den Details beschäftigen.
Es werden drei Funktionen/Methoden besprochen, nämlich `concat`, `merge` und `join`. Was unterscheidet diese *grundsätzlich*?
- Mit **`concat`** werden DataFrames aneinandergereiht ("gestapelt"), entweder untereinander (`axis=0`) oder nebeneinander (`axis=1`). 


- Eine weitere Möglichkeit, DataFrames zu kombinieren, besteht darin, *in jedem Dataset Spalten zu verwenden, die gemeinsame Werte enthalten (eine allgemeine eindeutige ID)*. Hierzu verwenden wir grundsätzlich die Funktion **`merge`**. 
    - Die Kombination von DataFrames mit einem gemeinsamen Feld heisst *"Joining"*. 
    - Die Spalten mit den gemeinsamen Werten heissen "Join Key(s)". 
    - Das Verbinden von DataFrames auf diese Weise ist oft nützlich, wenn ein DataFrame eine "Nachschlagetabelle" ist, die zusätzliche Daten enthält, die wir in die andere einschliessen möchten (*"many-to-one merges"*).
    - Dieser Prozess zum Verknüpfen von Tabellen ähnelt dem, den wir mit Tabellen in SQL-Datenbanken ausführen.


- *Falls die gemeinsamen Werte in den Indizes vorliegen*, kann man statt `merge` (mit den Argumenten `left_index=True`, `right_index=True`) die Methode **`join`** verwenden.

### Aneinanderreihen: `concat`
Mit `concat` werden einfach die Zeilen (oder Spalten, falls `axis=1`) von DataFrames aneinandergereiht bzw. gestapelt. *Dabei können Duplikate entstehen*.

In [ ]:
# Beispieldaten generieren:
umsatz_df1 = Umsatz_df.loc[:2019]
umsatz_df1

In [ ]:
# Weitere Beispieldaten generieren:
umsatz_df2 = Umsatz_df.loc[2020:]
umsatz_df2

**Zeilen anhängen:**

![](Abbildungen/concat_rows.png)

In [ ]:
pd.concat([umsatz_df1, umsatz_df2])

Man beachte, dass eine *Liste* mit DataFrames übergeben werden muss.

**Spalten anhängen:**

![](Abbildungen/concat_columns.png)

In [ ]:
# Beispieldaten generieren:
Umsatz_df2 = Umsatz_df.copy()
Umsatz_df2.rename({'Filiale_A': 'Filiale_C', 
                   'Filiale_B': 'Filiale_D'},
                   axis=1, inplace=True)
Umsatz_df2

In [ ]:
pd.concat([Umsatz_df, Umsatz_df2], axis=1)

Spalten werden flexibler und sicherer (korrekt ausgerichtet) mit `merge` oder `join` einem DataFrame hinzugefügt. Damit befasst sich der nächste Abschnitt.

### DataFrame im Datenbankstil verknüpfen: `merge` und `join`
- *Merge*- oder *join*-Operationen kombinieren Datasets, *indem sie Zeilen mit einem oder mehreren Schlüsseln verknüpfen*. 
- Diese Operationen sind für *relationale Datenbanken* (z.B. SQL-basiert) von zentraler Bedeutung. 
- Die `merge`-Funktion in Pandas ist der Haupteinstiegspunkt für die Verwendung dieser Algorithmen.

#### Verknüpfen: `merge`

![](Abbildungen/combine_data_sets.png)

In [ ]:
# Beispieldaten erzeugen:
adf = pd.DataFrame({'x1': ['A', 'B', 'C'], 'x2': [1, 2, 3]})
bdf = pd.DataFrame({'x1': ['A', 'B', 'D'], 'x3': ['T', 'F', 'T']})
adf

In [ ]:
bdf

Zuerst mergen wir die DataFrames `adf` und `bdf` über die Spalte/Variable `x1`. Mit dem Argument `how=left` werden die Daten an der Variable (hier `x1`) des "linken" (ersten) DataFrame ausgerichtet. Werte der Merge-Variablen (`x1`), die *nur* im rechten (zweiten) DataFrame vorkommen, werden nicht übernommen.

![](Abbildungen/merge_left.png)

In [ ]:
pd.merge(adf, bdf, how='left', on='x1')

In [ ]:
# Oder als Methode mit gleichem Ergebnis:
adf.merge(bdf, how='left', on='x1')

C in Merge-Spalte `x1` gibt es nur im ersten (linken) DataFrame, so dass die Variable `x3` an der Stelle ein `NaN` enthält. D in Merge-Spalte `x1` gibt es nur im zweiten (rechten) DataFrame und wird bei `how='left'` nicht übernommen.

Man kann auch an der Merge-Variable (`x1`) des rechten (zweiten) DataFrame ausrichten.

![](Abbildungen/merge_right.png)

In [ ]:
pd.merge(adf, bdf, how='right', on='x1')

Mit dem Argument `how=inner` werden nur Zeilen über die Merge-Variable verbunden, bei denen die Ausprägungen der Merge-Variable in beiden DataFrames vorkommen (Schnittmenge).

![](Abbildungen/merge_inner.png)

In [ ]:
pd.merge(adf, bdf, how='inner', on='x1')

Mit dem Argument `how=outer` werden Zeilen über die Merge-Variable verbunden, bei denen die Ausprägungen der Merge-Variable in einem der beiden DataFrames vorkommen (Vereinigungsmenge).

![](Abbildungen/merge_outer.png)

In [ ]:
pd.merge(adf, bdf, how='outer', on='x1')

**Many-to-one Merges:**
Das Verbinden von DataFrames ist oft nützlich, wenn ein DataFrame eine "Nachschlagetabelle" ist, die zusätzliche Daten enthält, die wir in die andere einschliessen möchten (*"many-to-one merges"*).

In [ ]:
Stammdaten = pd.DataFrame({'Person': list('ABCD'),
                           'Arbeitgeber': ['ZHAW', 'UBS', 'UBS', 'ZHAW']},
                           columns=['Person', 'Arbeitgeber'])
Stammdaten

In [ ]:
Nachschlagetabelle = pd.DataFrame({'Arbeitgeber': ['UBS', 'ZHAW'],
                       'Trägerschaft': ['privat', 'öffentlich'],
                       'Branche': ['Bank', 'Hochschule']})
Nachschlagetabelle

In [ ]:
Stammdaten.merge(Nachschlagetabelle, on='Arbeitgeber')

**Kontrollfragen:**

In [ ]:
# Gegeben:
df1 = pd.DataFrame({'x': ['A', 'B', 'C'],
                    'y': [1, 2, 3]})
df1

In [ ]:
# Gegeben ein weiteres DataFrame:
df2 = pd.DataFrame({'x': ['A', 'A', 'B'],
                    'z': [11, 12, 13]})
df2

In [ ]:
# Frage 1: Was ist der Output?
pd.merge(df1, df2, on='x', how='outer')

In [ ]:
# Frage 2: Was ist der Output?
pd.merge(df1, df2, on='x', how='right')

#### Verknüpfen am Index: `join`

Oft ist die Information, die zum Verbinden zweier DataFrames nötig ist, im Index der DataFrames enthalten.

In [ ]:
# Beispieldaten erzeugen:
adf.set_index('x1', inplace=True)
adf

In [ ]:
bdf.set_index('x1', inplace=True)
bdf

Mit `merge()` und den Argumenten `left_index=True` und `right_index=True` können die zwei DataFrames verbunden werden. 

In [ ]:
adf.merge(bdf, left_index=True, right_index=True, how='inner')

Einfacher geht es mit der `join`-Methode.

In [ ]:
adf.join(bdf, how='inner')

**Kontrollfrage:**

In [ ]:
# Gegeben:
Umsatz_df

In [ ]:
# Gegeben:
Umsatz_df2

In [ ]:
# Frage: Was ist der Output?
Umsatz_df.join(Umsatz_df2)

- Wir haben damit die wichtigsten Funktionalitäten bezüglich Kombinieren von Daten-Sets *im Grundsatz* besprochen.
- Das Lehrmittel vertieft die Ausführungen und bespricht insbesondere auch:
    - Many-to-many Merges
    - Merge bei hierarchischen Indizes

### Rekapitulation: Was ist nochmals der Unterschied zwischen `merge`, `join` und `concat`?
- `concat` wird verwendet, um ein (oder mehrere) DataFrame(s) an ein anderes untereinander anzuhängen (oder auch seitwärts, falls `axis=1` gesetzt ist).
- Ein wesentlicher Unterschied zu `concat` besteht darin, dass `merge` verwendet wird, um zwei (oder mehr) DataFrames auf der *Basis von Werten gemeinsamer Spalten* zu verbinden (Indizes können auch verwendet werden, `left_index=True` und/oder `right_index=True`).
- `join` kann verwendet werden, um zwei DataFrames *auf der Basis des Index* zusammenzuführen (was einfacher ist, als `merge` mit der Option `left_index=True` und/oder `right_index=True` zu verwenden).

## Umformen und Transponieren (Reshaping and Pivoting)

### Umformen bei hierarchischem Index
- Die hierarchische Indexierung bietet eine konsistente Möglichkeit, Daten in einem DataFrame neu anzuordnen. 
- Es gibt zwei Hauptaktionen (welche oben bereits kurz eingeführt wurden): 
    - **`stack`**: Rotiert oder schwenkt von den Spalten in den Daten zu den Zeilen
    - **`unstack`**: Rotiert von den Zeilen in die Spalten

In [ ]:
# Beispieldaten:
Umsatz_df

In [ ]:
Long_format = Umsatz_df.stack()
Long_format

In [ ]:
Wide_format = Long_format.unstack()
Wide_format

- Standardmässig wird die innerste Ebene "entstapelt" (dasselbe gilt für `stack`). 
- Sie können eine andere Ebene entstapeln, indem Sie eine Ebenennummer oder Namen übergeben:

### Pivoting vom "long"- zum "wide"-Format
- Eine gängige Möglichkeit, mehrere Zeitreihen in Datenbanken (z. B. MySQL) und CSV-Dateien zu speichern, ist das so genannte "Long"- oder "Stacked"-Format. 
- Beispiel:

In [ ]:
long_data = pd.read_csv('../weitere_Daten/long_data.csv')
long_data.head()

- In manchen Fällen ist es schwieriger, mit den Daten in diesem Format zu arbeiten. Vielleicht bevorzugen wir ein DataFrame mit einer Spalte pro Variable, der durch Zeitstempel in der Datumsspalte indiziert wird. 
- Die DataFrame-Methode **`pivot`** führt genau diese Transformation durch:

![](Abbildungen/pivot.png)

In [ ]:
wide_data = long_data.pivot(index='date', columns='variable', values='value')
wide_data.head()

- Die ersten beiden übergebenen Werte (wenn nicht wie oben explizit angegeben) sind die Spalten, die jeweils als Zeilen- und Spaltenindex verwendet werden, und schliesslich eine optionale Wertespalte, um das DataFrame zu füllen. 

### `pivot` vs. `unstack`
**Beachten Sie, dass `pivot` dem Erstellen eines hierarchischen Indexes mit `set_index` gefolgt von einem Aufruf von `unstack` entspricht:**

In [ ]:
# Zur Erinnerung:
long_data.head()

In [ ]:
# Hierarchischer Index erstellen:
df_HI = long_data.set_index(['date', 'variable'])    
df_HI.head()

In [ ]:
# Nun unstack; gleiches Ergebnis wie mit pivot oben:
df_HI.unstack('variable').head()

## Fazit
- Wir haben uns nun wichtige Pandas-Grundlagen für den Datenimport, -säuberung und -reorganisation erarbeitet.
- Im nächsten Kapitel werden wir uns mit der *Datenvisualisierung* genauer befassen. 
- Wir werden später auf Pandas zurückkommen, wenn wir fortgeschrittenere Analysen (Gruppierungen, Zeitreihen) diskutieren.